In [7]:
%pip install pandas
%pip install requests  # Corrigido para 'requests'
%pip install lxml
!pip install requests


import pandas as pd
import requests

# URL da página com as tabelas
url = 'https://capital.sp.gov.br/web/mobilidade/w/institucional/sptrans/acesso_a_informacao/245234'

# Baixar o conteúdo da página
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    tabelas = pd.read_html(response.text, header=1)  # Define a segunda linha como cabeçalho

    # Dicionário para armazenar os DataFrames
    dfs = {}

    # Criar um DataFrame para cada tabela e armazenar no dicionário
    for i, tabela in enumerate(tabelas):
        nome_df = f'df{i+1}'
        dfs[nome_df] = tabela
else:
    print(f"Erro ao acessar a página: {response.status_code}")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


C:\Users\joth1\AppData\Local\Temp\ipykernel_39012\1350871416.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabelas = pd.read_html(response.text, header=1)  # Define a segunda linha como cabeçalho


In [19]:
import numpy as np

#### Como funciona cada coluna ?

## Descrição das Colunas

- **MÊS**: Número do mês (1 a 12).
- **Grupo Estrutural**: Numero de passageiros transportados em onibus do tipo de linha (ex: principal, alimentadora dos centros e afins).
- **Grupo Local de Articulação Regional**: Numero de passageiros transportados em linhas regionais
- **Grupo Local de Distribuição**: Numero de passageiros transportados em linhas regionais

- **Total**: Total de passageiros ou unidades transportadas.
- **particao**: Segmentação dos dados, geralmente por ano ou período.
- **Período**: Intervalo de tempo de coleta (ex: 2024).

In [8]:
for i in range(1, 13, 1):
  print('mundo')
  df = dfs[f'df{i}']

  #  Crio a partição da base
  if i == 1:
    valor = 2024
  else:
    valor = valor - 1
  df['particao'] = valor
  if i == 1:
    df_final = df.copy()
  else:
    
    # Renomeando e substituindo as colunas com os valores e apagando as antigas
    if 'Estrutural' in df.columns:
        df['Grupo Estrutural'] = df['Estrutural']
        df.drop(columns=['Estrutural'], inplace=True)

    if 'Local' in df.columns:
        df['Grupo Local de Articulação Regional'] = df['Local']
        df.drop(columns=['Local'], inplace=True)

    if 'Sistema' in df.columns:
        df['Total'] = df['Sistema']
        df.drop(columns=['Sistema'], inplace=True)

    if 'Subsistema Estrutural' in df.columns:
        df['Grupo Estrutural'] = df['Subsistema Estrutural']
        df.drop(columns=['Subsistema Estrutural'], inplace=True)

    if 'Subsistema Local' in df.columns:
        df['Grupo Local de Articulação Regional'] = df['Subsistema Local']
        df.drop(columns=['Subsistema Local'], inplace=True)
        # Adiciona o DataFrame na lista

          # Verifica se as colunas existem
        # Verifica se a coluna 'Grupo Local de Distribuição' existe
    if 'Grupo Local de Distribuição' in df.columns:
        print('entrou')
        # Substituindo valores ausentes (NaN) por 0, mas não mexendo em valores válidos
        df['Grupo Local de Distribuição'] = df['Grupo Local de Distribuição'].fillna(0)
        df['Grupo Local de Articulação Regional'] = df['Grupo Local de Articulação Regional'].fillna(0)


        # Removendo os pontos (separadores de milhar) e tratando o "-" como 0
        df['Grupo Local de Articulação Regional'] = df['Grupo Local de Articulação Regional'].replace({'.': '', '-': '0'}, regex=True)
        df['Grupo Local de Distribuição'] = df['Grupo Local de Distribuição'].replace({'.': '', '-': '0'}, regex=True)

        # Removendo a coluna 'Grupo Local de Distribuição' já que foi somada
        df.drop(columns=['Grupo Local de Distribuição'], inplace=True)


    # Decrementa o valor da partição
    valor -= 1
    df_final = pd.concat([df_final, df])

mundo
mundo
entrou
mundo
entrou
mundo
entrou
mundo
entrou
mundo
entrou
mundo
mundo
mundo
mundo
mundo
mundo


### Tratamento das variaveis

In [9]:
# Remove os pontos (separadores de milhar)
df_final['Grupo Local de Distribuição'] = df_final['Grupo Local de Distribuição'].str.replace('.', '', regex=False)
df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'].str.replace('.', '', regex=False)
df_final['Total'] = df_final['Total'].str.replace('.', '', regex=False)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].str.replace('.', '', regex=False)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].str.replace('-', '', regex=False)


# Substitui valores ausentes (NaN) e strings vazias ('') por '0'
df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'].fillna('0').replace('', '0')
df_final['Grupo Local de Distribuição'] = df_final['Grupo Local de Distribuição'].fillna('0').replace('', '0')
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].fillna('0').replace('', '0')

In [11]:
df_final

,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período
0,Jan/24,52192455,105735318,157927773,2024,NaN
1,Fev/24,55127009,112459029,167586038,2024,NaN
2,Mar/24,59960019,122934281,182894300,2024,NaN
3,Abr/24,62076320,126957579,189033899,2024,NaN
4,Mai/24,60742151,124806116,185548267,2024,NaN
...,...,...,...,...,...,...
8,NaN,140055474,109239653,249295127,2003,Setembro
9,NaN,150181032,116184406,266365438,2003,Outubro
10,NaN,136361483,107257315,243618798,2003,Novembro
11,NaN,128767687,103745163,232512850,2003,Dezembro


### Trato as colunas de região e Onibus de transporte até o centro

In [17]:

# Tenta converter as colunas para tipo float
df_final['Grupo Local de Articulação Regional'] = pd.to_numeric(df_final['Grupo Local de Articulação Regional'],errors='coerce')
# df_final['Grupo Local de Distribuição'] = pd.to_numeric(df_final['Grupo Local de Distribuição'], errors='coerce')

# # Soma das colunas
# df_final['Grupo Local de Articulação Regional'] = df_final['Grupo Local de Articulação Regional'] + df_final['Grupo Local de Distribuição']

# Remove a coluna 'Grupo Local de Distribuição' após a soma
# df_final.drop(columns=['Grupo Local de Distribuição'], inplace=True)
df_final['Total'] = df_final['Total'].fillna('0').astype(int)
df_final['Grupo Estrutural'] = df_final['Grupo Estrutural'].fillna('0').astype(int)

# Exibe o DataFrame final
display(df_final)

,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período
0,Jan/24,52192455,105735318,157927773,2024,NaN
1,Fev/24,55127009,112459029,167586038,2024,NaN
2,Mar/24,59960019,122934281,182894300,2024,NaN
3,Abr/24,62076320,126957579,189033899,2024,NaN
4,Mai/24,60742151,124806116,185548267,2024,NaN
...,...,...,...,...,...,...
8,NaN,140055474,109239653,249295127,2003,Setembro
9,NaN,150181032,116184406,266365438,2003,Outubro
10,NaN,136361483,107257315,243618798,2003,Novembro
11,NaN,128767687,103745163,232512850,2003,Dezembro


In [ ]:
# Dicionário de meses
meses = {
    'Jan': 'Janeiro', 'Fev': 'Fevereiro', 'Mar': 'Março', 'Abr': 'Abril', 
    'Mai': 'Maio', 'Jun': 'Junho', 'Jul': 'Julho', 'Ago': 'Agosto', 
    'Set': 'Setembro', 'Out': 'Outubro', 'Nov': 'Novembro', 'Dez': 'Dezembro'
}

df_final['sep'] = df_final['MÊS'].str.split('/').str[0]

# Substituindo a abreviação pelo nome completo do mês
df_final['Período'] = np.where(df_final['sep'].isin(meses.keys()),
                           df_final['sep'].map(meses),
                           df_final['Período'])

# Exibindo o resultado
display(df_final)


,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,sep
0,Jan/24,52192455,105735318,157927773,2024,NaN,Jan
1,Fev/24,55127009,112459029,167586038,2024,NaN,Fev
2,Mar/24,59960019,122934281,182894300,2024,NaN,Mar
3,Abr/24,62076320,126957579,189033899,2024,NaN,Abr
4,Mai/24,60742151,124806116,185548267,2024,NaN,Mai
...,...,...,...,...,...,...,...
8,NaN,140055474,109239653,249295127,2003,Setembro,NaN
9,NaN,150181032,116184406,266365438,2003,Outubro,NaN
10,NaN,136361483,107257315,243618798,2003,Novembro,NaN
11,NaN,128767687,103745163,232512850,2003,Dezembro,NaN


,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,sep
0,Jan/24,52192455,105735318,157927773,2024,Janeiro,Jan
1,Fev/24,55127009,112459029,167586038,2024,Fevereiro,Fev
2,Mar/24,59960019,122934281,182894300,2024,Março,Mar
3,Abr/24,62076320,126957579,189033899,2024,Abril,Abr
4,Mai/24,60742151,124806116,185548267,2024,Maio,Mai
...,...,...,...,...,...,...,...
8,NaN,140055474,109239653,249295127,2003,Setembro,NaN
9,NaN,150181032,116184406,266365438,2003,Outubro,NaN
10,NaN,136361483,107257315,243618798,2003,Novembro,NaN
11,NaN,128767687,103745163,232512850,2003,Dezembro,NaN


In [21]:
# Criando uma nova coluna com as três primeiras letras da coluna 'MÊS'
df_final['MÊS_abrev'] = df_final['Período'].str.slice(0, 3)

df_final['MÊS'] = df_final['MÊS_abrev'].astype(str) + '/' + df_final['particao'].astype(str)
display(df_final)

df_final.drop(columns=['sep'], inplace=True)


,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,sep,MÊS_abrev
0,Jan/2024,52192455,105735318,157927773,2024,Janeiro,Jan,Jan
1,Fev/2024,55127009,112459029,167586038,2024,Fevereiro,Fev,Fev
2,Mar/2024,59960019,122934281,182894300,2024,Março,Mar,Mar
3,Abr/2024,62076320,126957579,189033899,2024,Abril,Abr,Abr
4,Mai/2024,60742151,124806116,185548267,2024,Maio,Mai,Mai
...,...,...,...,...,...,...,...,...
8,Set/2003,140055474,109239653,249295127,2003,Setembro,NaN,Set
9,Out/2003,150181032,116184406,266365438,2003,Outubro,NaN,Out
10,Nov/2003,136361483,107257315,243618798,2003,Novembro,NaN,Nov
11,Dez/2003,128767687,103745163,232512850,2003,Dezembro,NaN,Dez


In [22]:
df_final

,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,MÊS_abrev
0,Jan/2024,52192455,105735318,157927773,2024,Janeiro,Jan
1,Fev/2024,55127009,112459029,167586038,2024,Fevereiro,Fev
2,Mar/2024,59960019,122934281,182894300,2024,Março,Mar
3,Abr/2024,62076320,126957579,189033899,2024,Abril,Abr
4,Mai/2024,60742151,124806116,185548267,2024,Maio,Mai
...,...,...,...,...,...,...,...
8,Set/2003,140055474,109239653,249295127,2003,Setembro,Set
9,Out/2003,150181032,116184406,266365438,2003,Outubro,Out
10,Nov/2003,136361483,107257315,243618798,2003,Novembro,Nov
11,Dez/2003,128767687,103745163,232512850,2003,Dezembro,Dez


In [49]:
df_final2 = df_final.loc[ (df_final['Período'] != 'Total') & (df_final['Período'].notna()) ]
df_final2

,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,MÊS_abrev
0,Jan/2024,52192455,105735318,157927773,2024,Janeiro,Jan
1,Fev/2024,55127009,112459029,167586038,2024,Fevereiro,Fev
2,Mar/2024,59960019,122934281,182894300,2024,Março,Mar
3,Abr/2024,62076320,126957579,189033899,2024,Abril,Abr
4,Mai/2024,60742151,124806116,185548267,2024,Maio,Mai
...,...,...,...,...,...,...,...
7,Ago/2003,146954904,114528569,261483473,2003,Agosto,Ago
8,Set/2003,140055474,109239653,249295127,2003,Setembro,Set
9,Out/2003,150181032,116184406,266365438,2003,Outubro,Out
10,Nov/2003,136361483,107257315,243618798,2003,Novembro,Nov


In [50]:
cubo_grupo = df_final2.groupby('particao').agg(
    qtd=('MÊS', 'count')
    ,media=('Grupo Estrutural', 'mean')
    ,media2=('Total', 'mean')
)
display(cubo_grupo)

,qtd,media,media2
particao,,,
2003,12,1.362653e+08,2.436844e+08
2005,12,1.377168e+08,2.434064e+08
2007,12,1.388466e+08,2.413090e+08
2009,12,1.392739e+08,2.429453e+08
2011,12,1.358841e+08,2.386893e+08
2013,12,1.315073e+08,2.332430e+08
2015,12,8.433283e+07,2.198492e+08
2017,12,4.246357e+07,1.302711e+08
2019,12,4.577052e+07,1.395440e+08


In [51]:
tst = df_final2.query('particao == 2003')
display(tst)

tst = df_final2.query('particao == 2024')
display(tst)

,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,MÊS_abrev
0,Jan/2003,122434713,96949805,219384518,2003,Janeiro,Jan
1,Fev/2003,121443684,96866447,218310131,2003,Fevereiro,Fev
2,Mar/2003,139660959,110954359,250615318,2003,Março,Mar
3,Abr/2003,145045354,112661569,257706923,2003,Abril,Abr
4,Mai/2003,141588617,111199329,252787946,2003,Maio,Mai
5,Jun/2003,133504167,105616178,239120345,2003,Junho,Jun
6,Jul/2003,129185242,103826356,233011598,2003,Julho,Jul
7,Ago/2003,146954904,114528569,261483473,2003,Agosto,Ago
8,Set/2003,140055474,109239653,249295127,2003,Setembro,Set
9,Out/2003,150181032,116184406,266365438,2003,Outubro,Out


,MÊS,Grupo Estrutural,Grupo Local de Articulação Regional,Total,particao,Período,MÊS_abrev
0,Jan/2024,52192455,105735318,157927773,2024,Janeiro,Jan
1,Fev/2024,55127009,112459029,167586038,2024,Fevereiro,Fev
2,Mar/2024,59960019,122934281,182894300,2024,Março,Mar
3,Abr/2024,62076320,126957579,189033899,2024,Abril,Abr
4,Mai/2024,60742151,124806116,185548267,2024,Maio,Mai
5,Jun/2024,59334975,122106752,181441727,2024,Junho,Jun
6,Jul/2024,57851224,116699854,174551078,2024,Julho,Jul
7,Ago/2024,63180602,129337500,192518102,2024,Agosto,Ago
8,Set/2024,60958004,125572354,186530358,2024,Setembro,Set
9,Out/2024,63947289,130959003,194906292,2024,Outubro,Out


In [ ]:
# 219.384.518
# 150.555.599

SyntaxError: invalid syntax (2639776141.py, line 1)

In [11]:
# Supondo que você já tenha o DataFrame `df`
df_final.to_csv(r'C:\Users\Administrador\Music\Project-SPTrans-Data-Analysis\bases_salvas\dados_populacao_transportada.csv', index=False)